In [1]:
import matplotlib.pyplot as plt
from scipy import stats
import random
from matplotlib.patches import Circle
from matplotlib.colors import ListedColormap
import numpy

In [ ]:
def generate_weighted_voronoi(sites, weights, grid_size, bbox):
    """
    Generates an Additively Weighted Voronoi Diagram using a pixel-based approach.
    Modified for DRG cellular morphology modeling.
    """
    xmin, xmax, ymin, ymax = bbox
    
    # Create the grid of pixels
    x_coords = np.linspace(xmin, xmax, grid_size)
    y_coords = np.linspace(ymin, ymax, grid_size)
    xx, yy = np.meshgrid(x_coords, y_coords)
    
    # Initialize maps
    min_dist_map = np.full((grid_size, grid_size), np.inf)
    voronoi_map = np.zeros((grid_size, grid_size), dtype=int)
    
    # Calculate the owner of each pixel
    num_sites = len(sites)
    for i in range(num_sites):
        sx, sy = sites[i]
        w = weights[i]
        
        # Calculate Euclidean distance from every pixel to the current site
        dist_sq = (xx - sx)**2 + (yy - sy)**2
        dist = np.sqrt(dist_sq)
        
        # Calculate the additively weighted distance
        weighted_dist = dist - w
        
        # Find pixels where the current site is closer than any previous site
        mask = weighted_dist < min_dist_map
        
        # Update the maps
        min_dist_map[mask] = weighted_dist[mask]
        voronoi_map[mask] = i
        
    return voronoi_map

def sample_radii_from_distribution(n_cells, distribution='lognormal', **kwargs):
    """
    Sample cell radii from various distributions commonly observed in biological systems.
    
    Args:
        n_cells (int): Number of cells to generate
        distribution (str): Type of distribution ('lognormal', 'gamma', 'normal', 'truncated_normal', 'custom')
        **kwargs: Parameters for the distribution
        
    Returns:
        np.ndarray: Array of sampled radii
    """
    if distribution == 'lognormal':
        # Default parameters for DRG neurons (adjust based on your data)
        mu = kwargs.get('mu', 1.5)  # log-scale mean
        sigma = kwargs.get('sigma', 0.5)  # log-scale std
        radii = np.random.lognormal(mu, sigma, n_cells)
        
    elif distribution == 'gamma':
        # Shape and scale parameters
        shape = kwargs.get('shape', 2.0)
        scale = kwargs.get('scale', 10.0)
        radii = np.random.gamma(shape, scale, n_cells)
        
    elif distribution == 'normal':
        mean = kwargs.get('mean', 15.0)
        std = kwargs.get('std', 5.0)
        radii = np.random.normal(mean, std, n_cells)
        radii = np.clip(radii, 1.0, None)  # Ensure positive radii
        
    elif distribution == 'truncated_normal':
        # Truncated normal distribution - ideal for biological measurements
        mean = kwargs.get('mean', 15.0)
        std = kwargs.get('std', 5.0)
        lower_bound = kwargs.get('lower_bound', 0.0)  # Minimum cell size
        upper_bound = kwargs.get('upper_bound', None)  # Maximum cell size (None = no upper bound)
        
        # Convert to scipy.stats parameters
        a = (lower_bound - mean) / std if lower_bound is not None else -np.inf
        b = (upper_bound - mean) / std if upper_bound is not None else np.inf
        
        # Sample from truncated normal
        radii = stats.truncnorm.rvs(a, b, loc=mean, scale=std, size=n_cells)
        
    elif distribution == 'custom':
        # Pass a callable function that returns n_cells samples
        sampler = kwargs.get('sampler')
        if sampler is None:
            raise ValueError("For custom distribution, provide 'sampler' function")
        radii = sampler(n_cells)
        
    else:
        raise ValueError(f"Unknown distribution: {distribution}")
    
    return radii

def generate_sites_poisson_disk(bbox, min_distance, max_attempts=1000):
    """
    Generate sites using Poisson disk sampling for more natural spatial distribution.
    
    Args:
        bbox (tuple): Bounding box (xmin, xmax, ymin, ymax)
        min_distance (float): Minimum distance between sites
        max_attempts (int): Maximum attempts to place each point
        
    Returns:
        np.ndarray: Array of site coordinates
    """
    xmin, xmax, ymin, ymax = bbox
    width, height = xmax - xmin, ymax - ymin
    
    # Grid for efficient neighbor checking
    cell_size = min_distance / np.sqrt(2)
    grid_width = int(np.ceil(width / cell_size))
    grid_height = int(np.ceil(height / cell_size))
    grid = np.full((grid_height, grid_width), -1, dtype=int)
    
    sites = []
    active_list = []
    
    # Start with a random point
    first_point = np.array([
        xmin + random.random() * width,
        ymin + random.random() * height
    ])
    sites.append(first_point)
    active_list.append(0)
    
    # Add to grid
    gx = int((first_point[0] - xmin) / cell_size)
    gy = int((first_point[1] - ymin) / cell_size)
    grid[gy, gx] = 0
    
    while active_list:
        # Pick random active point
        active_idx = random.randint(0, len(active_list) - 1)
        point_idx = active_list[active_idx]
        point = sites[point_idx]
        
        found = False
        for _ in range(max_attempts):
            # Generate candidate point
            angle = random.random() * 2 * np.pi
            radius = min_distance * (1 + random.random())
            candidate = point + radius * np.array([np.cos(angle), np.sin(angle)])
            
            # Check bounds
            if not (xmin <= candidate[0] <= xmax and ymin <= candidate[1] <= ymax):
                continue
            
            # Check grid position
            gx = int((candidate[0] - xmin) / cell_size)
            gy = int((candidate[1] - ymin) / cell_size)
            
            # Check neighbors in grid
            valid = True
            for dy in range(-2, 3):
                for dx in range(-2, 3):
                    nx, ny = gx + dx, gy + dy
                    if 0 <= nx < grid_width and 0 <= ny < grid_height:
                        neighbor_idx = grid[ny, nx]
                        if neighbor_idx >= 0:
                            dist = np.linalg.norm(candidate - sites[neighbor_idx])
                            if dist < min_distance:
                                valid = False
                                break
                if not valid:
                    break
            
            if valid:
                sites.append(candidate)
                active_list.append(len(sites) - 1)
                grid[gy, gx] = len(sites) - 1
                found = True
                break
        
        if not found:
            active_list.pop(active_idx)
    
    return np.array(sites)


def plot_voronoi_tessellation(voronoi_map, sites, weights, bbox, grid_size, 
                            show_sites=True, show_weights=False, title="DRG-like Voronoi Tessellation"):
    """
    Plot the weighted Voronoi tessellation with optimal coloring to distinguish adjacent cells.
    """
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    
    n_sites = len(sites)
    
    # Find neighboring relationships
    neighbors = find_voronoi_neighbors(voronoi_map, n_sites)
    
    # Color the graph to ensure adjacent cells have different colors
    coloring = greedy_graph_coloring(neighbors)
    n_colors_used = len(set(coloring.values()))
    
    print(f"Used {n_colors_used} colors for {n_sites} cells")
    
    # Create colormap with distinct colors
    cmap = create_distinct_colormap(n_colors_used)
    #cmap = 'magma'
    # Create a recolored voronoi map
    colored_voronoi_map = np.zeros_like(voronoi_map)
    for cell_idx, color_idx in coloring.items():
        colored_voronoi_map[voronoi_map == cell_idx] = color_idx
    
    # Plot the tessellation with distinct colors
    xmin, xmax, ymin, ymax = bbox
    ax.imshow(colored_voronoi_map, extent=[xmin, xmax, ymin, ymax], 
              cmap=cmap, origin='lower', alpha=0.8, vmin=0, vmax=n_colors_used-1)
    
    # Add black borders between cells for better definition
    from scipy import ndimage
    edges = ndimage.sobel(voronoi_map.astype(float))
    edges = edges > 0
    ax.contour(edges, extent=[xmin, xmax, ymin, ymax], colors='black', 
               linewidths=0.5, alpha=0.6, levels=[0.5])
    
    # Plot sites
    if show_sites:
        ax.scatter(sites[:, 0], sites[:, 1], c='black', s=30, zorder=5, 
                  edgecolors='white', linewidth=1)
    
    # Plot weight circles (original intended radii)
    if show_weights:
        for i, (site, weight) in enumerate(zip(sites, weights)):
            circle = Circle(site, weight, fill=False, edgecolor='red', 
                          linewidth=1.5, alpha=0.9, linestyle='--')
            ax.add_patch(circle)
    
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_aspect('equal')
    ax.set_title(f"{title}\n({n_colors_used} colors used)")
    ax.grid(True, alpha=0.3)
    
    return fig, ax

def find_voronoi_neighbors(voronoi_map, n_sites):
    """
    Find neighboring relationships between Voronoi cells by checking adjacent pixels.
    
    Args:
        voronoi_map (np.ndarray): 2D array with cell indices
        n_sites (int): Number of sites/cells
        
    Returns:
        dict: Dictionary where keys are cell indices and values are sets of neighboring cell indices
    """
    neighbors = {i: set() for i in range(n_sites)}
    
    # Check 4-connectivity (up, down, left, right)
    height, width = voronoi_map.shape
    
    for y in range(height - 1):
        for x in range(width - 1):
            current_cell = voronoi_map[y, x]
            
            # Check right neighbor
            if x < width - 1:
                right_cell = voronoi_map[y, x + 1]
                if current_cell != right_cell:
                    neighbors[current_cell].add(right_cell)
                    neighbors[right_cell].add(current_cell)
            
            # Check bottom neighbor  
            if y < height - 1:
                bottom_cell = voronoi_map[y + 1, x]
                if current_cell != bottom_cell:
                    neighbors[current_cell].add(bottom_cell)
                    neighbors[bottom_cell].add(current_cell)
    
    return neighbors

def greedy_graph_coloring(neighbors, n_colors=None):
    """
    Color the graph using greedy coloring algorithm to ensure adjacent cells have different colors.
    
    Args:
        neighbors (dict): Dictionary of neighboring relationships
        n_colors (int): Maximum number of colors to use (None for automatic)
        
    Returns:
        dict: Dictionary mapping cell index to color index
    """
    n_sites = len(neighbors)
    if n_colors is None:
        # Use more colors than theoretically needed for better visual separation
        n_colors = min(12, max(6, n_sites // 3))
    
    coloring = {}
    
    # Sort nodes by degree (number of neighbors) in descending order
    # This heuristic often leads to better colorings
    nodes_by_degree = sorted(neighbors.keys(), key=lambda x: len(neighbors[x]), reverse=True)
    
    for node in nodes_by_degree:
        # Find colors used by neighbors
        used_colors = set()
        for neighbor in neighbors[node]:
            if neighbor in coloring:
                used_colors.add(coloring[neighbor])
        
        # Assign the first available color
        for color in range(n_colors):
            if color not in used_colors:
                coloring[node] = color
                break
        else:
            # If no color is available, use a random one (shouldn't happen with enough colors)
            coloring[node] = random.randint(0, n_colors - 1)
    
    return coloring

def create_distinct_colormap(n_colors):
    """
    Create a colormap with visually distinct colors.
    """
    if n_colors <= 12:
        # Use predefined distinct colors for small numbers
        distinct_colors = [
            '#FF6B6B',  # Red
            '#4ECDC4',  # Teal  
            '#45B7D1',  # Blue
            '#96CEB4',  # Green
            '#FECA57',  # Yellow
            '#FF9FF3',  # Pink
            '#54A0FF',  # Light blue
            '#5F27CD',  # Purple
            '#00D2D3',  # Cyan
            '#FF9F43',  # Orange
            '#10AC84',  # Sea green
            '#EE5A24'   # Red orange
        ]
        colors = distinct_colors[:n_colors]
    else:
        # For larger numbers, use HSV space for maximum distinction
        hues = np.linspace(0, 1, n_colors, endpoint=False)
        colors = plt.cm.hsv(hues)
    
    return ListedColormap(colors)

#@njit
def generate_weighted_voronoi_spaced(sites, weights, grid_size, bbox, space=0):
    """
    Generates an Additively Weighted Voronoi Diagram using a pixel-based approach,
    with spacing between cells.

    Args:
        sites (list of tuples): A list of (x, y) coordinates for the Voronoi sites.
        weights (list of floats): A list of weights corresponding to each site.
        grid_size (int): The number of pixels along one dimension of the grid.
        bbox (tuple): A tuple (xmin, xmax, ymin, ymax) defining the bounding box.
        space (float): The desired spacing between Voronoi cells. Pixels in this
                       space will be marked as 0.

    Returns:
        numpy.ndarray: A 2D array representing the Voronoi diagram.
    """
    xmin, xmax, ymin, ymax = bbox
    
    # Create the grid of pixels
    x_coords = np.linspace(xmin, xmax, grid_size)
    y_coords = np.linspace(ymin, ymax, grid_size)
    xx, yy = np.meshgrid(x_coords, y_coords)
    
    # Initialize maps
    min_dist_map = np.full((grid_size, grid_size), np.inf)
    voronoi_map = np.zeros((grid_size, grid_size), dtype=int)
    
    # Calculate the owner of each pixel
    num_sites = len(sites)
    for i in range(num_sites):
        sx, sy = sites[i]
        w = weights[i]
        
        # Calculate Euclidean distance from every pixel to the current site
        dist_sq = (xx - sx)**2 + (yy - sy)**2
        dist = np.sqrt(dist_sq)
        
        # Calculate the additively weighted distance
        weighted_dist = dist - w
        
        # Identify pixels where the new site is closer than the current winner
        update_mask = weighted_dist < min_dist_map

        # Identify contested pixels that should become space.
        # This is where the difference between the new distance and the old
        # minimum distance is less than the desired space.
        # This must be calculated *before* updating min_dist_map.
        if space > 0:
            space_mask = np.abs(min_dist_map - weighted_dist) < space
        else:
            space_mask = np.zeros_like(voronoi_map, dtype=bool)

        # Update the maps for the new winning pixels
        min_dist_map[update_mask] = weighted_dist[update_mask]
        voronoi_map[update_mask] = i + 1
        
        # Apply the space mask to zero out the contested regions
        voronoi_map[space_mask] = 0
            
    return voronoi_map

In [ ]:
if __name__ == __main__:
    bbox = (0, 250, 0, 250)  # Bounding box
    grid_size = 1500
    n_cells =25# Number of cells
    radius = 24.525
    std_radius = 12.612
    scale = grid_size/(bbox[1]-bbox[0])
    print("Generating DRG-like cellular tessellation...")
    
    target_radii = sample_radii_from_distribution(
        n_cells, 
        distribution='truncated_normal', 
        mean=radius,     # Mean radius in micrometers (adjust to your data)
        std=std_radius,       # Standard deviation (adjust to your data)
        lower_bound=3.0,  # Minimum biologically plausible cell radius
        upper_bound=80.0  # Maximum biologically plausible cell radius (optional)
    )
    
    print(f"Target radii: mean={np.mean(target_radii):.2f}, std={np.std(target_radii):.2f}")
    
    # Step 2: Generate sites with Poisson disk sampling for natural spacing
    min_distance = np.mean(target_radii) * 2  # Minimum distance between cell centers
    sites = generate_sites_poisson_disk(bbox, min_distance)
    
    
    # If we have fewer sites than desired, supplement with random placement
    if len(sites) < n_cells:
        print(f"Generated {len(sites)} sites, supplementing to {n_cells}")
        xmin, xmax, ymin, ymax = bbox
        additional_sites = []
        for _ in range(n_cells - len(sites)):
            # Simple random placement for remaining sites
            site = np.array([
                xmin + np.random.random() * (xmax - xmin),
                ymin + np.random.random() * (ymax - ymin)
            ])
            additional_sites.append(site)
        sites = np.vstack([sites, additional_sites])
    else:
        sites = sites[:n_cells]  # Take only the first n_cells sites
    
    # Use target radii as weights (these represent desired cell sizes)
    weights = target_radii
    
    print(f"Generated {len(sites)} sites")
    
    # Step 3: Generate weighted Voronoi tessellation
    # voronoi_map = generate_weighted_voronoi(sites, weights, grid_size, bbox)
    
    # # Step 5: Visualize results
    # fig1, ax1 = plot_voronoi_tessellation(
    #     voronoi_map, sites, weights, bbox, grid_size,
    #     show_sites=True, show_weights=True,
    #     title="DRG-like Weighted Voronoi Tessellation\n(Red dashed circles = target sizes)"
    # )
    
    
    # plt.show()
    voronoi_map = generate_weighted_voronoi_spaced(sites, weights, grid_size, bbox,space =2.3)
    np.save('mask.npy', np.where(voronoi_map==0,1,0))
    np.save('voronoi_sites.npy',sites)